In [ ]:
import socket
import os

server_running = False
connection_info = ""

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def server_pgm():
    global server_running
    global connection_info
    host = socket.gethostname()
    port = 5002

    received_files_folder = r"D:\server"
    ensure_directory_exists(received_files_folder)

    while server_running:
        server_socket = socket.socket()
        server_socket.bind((host, port))
        server_socket.listen(1)
        conn, address = server_socket.accept()
        connection_info = f"Connection from: {address}\n"
        
        option = conn.recv(1).decode()
        
        if option == "1":
            # Handling file retrieval
            file_name = conn.recv(128).decode()
            file_path = os.path.join(received_files_folder, file_name)
            if os.path.exists(file_path) and os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    file_contents = file.read()
                    conn.send(file_contents.encode())
            else:
                conn.send(b"404")

        elif option == "2":
            # Handling file upload
            file_name_length = conn.recv(3).decode()
            file_name = conn.recv(int(file_name_length)).decode()
            file_name_ext = file_name.split(".")[-1]
            if file_name.split(".")[0].strip() == "":
                conn.send(b"400")
            elif file_name_ext != "txt":
                conn.send(b"415")
            else:
                file_size = conn.recv(5).decode()
                file_path = os.path.join(received_files_folder, file_name)
                if os.path.exists(file_path):
                    conn.send(b"409")
                else:
                    with open(file_path, 'wb') as file:
                        i = 0
                        while i < int(file_size):
                            chunk = conn.recv(128)
                            if not chunk:
                                break
                            file.write(chunk)
                            i += 128
                    conn.send(b"201")

        elif option == "3":
            # Handling file append
            filename_length = int(conn.recv(3).decode())
            file_name = conn.recv(filename_length).decode()
            file_name_ext = file_name.split(".")[-1]
            if file_name.split(".")[0].strip() == "":
                conn.send(b"400")
            elif file_name_ext != "txt":
                conn.send(b"415")
            else:
                file_size = conn.recv(5).decode()
                file_path = os.path.join(received_files_folder, file_name)
                if os.path.exists(file_path):
                    with open(file_path, 'ab') as file:
                        i = 0
                        while i < int(file_size):
                            chunk = conn.recv(128)
                            if not chunk:
                                break
                            file.write(chunk)
                            i += 128
                    conn.send(b"200")
                else:
                    with open(file_path, 'wb') as file:
                        i = 0
                        while i < int(file_size):
                            chunk = conn.recv(128)
                            if not chunk:
                                break
                            file.write(chunk)
                            i += 128
                    conn.send(b"201")

        elif option == "4":
            # Handling file deletion
            selected_file = conn.recv(128).decode()
            selected_file_path = os.path.join(received_files_folder, selected_file)
            if os.path.exists(selected_file_path):
                try:
                    os.remove(selected_file_path)
                    conn.send(b"200")
                except FileNotFoundError:
                    conn.send(b"404")
            else:
                conn.send(b"404")

        conn.close()
        server_socket.close()
        print(connection_info)

def start_server():
    global server_running
    if not server_running:
        server_running = True
        server_pgm()

def stop_server():
    global server_running
    if server_running:
        server_running = False
        print("Server is stopped.")

if __name__ == "__main__":
    print("1. Start Server")
    print("2. Stop Server")
    choice = input("Enter your choice: ")
    if choice == "1":
        start_server()
    elif choice == "2":
        stop_server()
    else:
        print("Invalid choice")


1. Start Server
2. Stop Server
Enter your choice: 1
Connection from: ('192.168.1.7', 58946)

